In [1]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_boston

# from sklearn.template import skltemplate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from plot_classifier import plot_classifier
#from sklearn.plot_classifier import plot_classifier
%matplotlib inline
import sqlite3

In [13]:
## A local DB mtpj with whole image of Flights  16M rows
database = "mtpj.db"

conn = sqlite3.connect(database)

## For prove of concept use Flights_slim instead of Flights
sql_in="""SELECT cf.delay_pct as carrier_v, 
          --of.delay_pct as origin_v,
          df.delay_pct as dest_v,
          df.airport as dest,
          CAST(fl.crs_arr_hh24  as interger) crs_arr_hh24,
          hf.delay_pct  as                    hh24_v,
          CAST(crs_elapsed_time as interger) crs_elapsed_time,
          CAST(fl.fl_month      as interger) month,
          mf.delay_pct  as                   month_v,
          CAST(haul_type        as interger) haul_type,
          CAST(arr_delay_min_gy as interger) arr_delay_min_gy,
          CAST(arr_delay_min_bin as interger) arr_delay_min_bin,
          CAST(arr_delay_ind    as interger) arr_delay_ind,
          CAST(arr_delay        as interger) arr_delay
          FROM  FLIGHTS_slim fl, CARRIER_DELAY_F cf, AIRPORT_f df, AIRPORT_f of,
                HH24_F      hf, MONTH_F mf
          WHERE fl.mkt_unique_carrier = cf.mkt_unique_carrier
          AND   fl.dest               = df.airport
          AND   fl.origin             = of.airport
          AND   fl.crs_arr_hh24       = hf.crs_arr_hh24
          AND   fl.fl_month           = mf.fl_month
          AND   fl.cancelled          = '0.0'       --  cancelled = No
          AND   CAST(crs_elapsed_time as REAL) > 60
          ORDER BY RANDOM()
          LIMIT 160000;"""

df_stage = pd.read_sql(sql_in, con=conn)
conn.close()
df_stage
##df_stage=pd.read_csv('flights_f1.csv')

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,crs_elapsed_time,month,month_v,haul_type,arr_delay_min_gy,arr_delay_min_bin,arr_delay_ind,arr_delay
0,17.76531,38.01551,FLL,9,27.26231,157,1,33.52233,0,-15,-15,0,-15
1,9.88030,36.61623,SDF,16,36.13049,110,1,33.52233,0,45,45,1,35
2,11.70175,35.60278,MSY,22,42.37835,80,1,33.52233,0,-15,-15,0,-8
3,8.33723,44.79845,EWR,22,42.37835,166,1,33.52233,0,15,15,1,21
4,8.33723,36.41499,MCO,11,29.34927,141,1,33.52233,0,15,15,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,6.95741,36.12802,DSM,14,32.72667,85,1,33.52233,0,-15,-15,0,-2
159996,9.88030,33.54229,PHL,20,42.09990,88,1,33.52233,0,-45,-45,0,-31
159997,11.70175,37.05741,DAL,18,40.37584,90,1,33.52233,0,-15,-15,0,-14
159998,9.88030,39.75784,SFO,15,34.27183,408,1,33.52233,1,-45,-45,0,-41


In [14]:
#df=df_stage.dropna()
### or 
df1=df_stage.dropna()
df1


,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,crs_elapsed_time,month,month_v,haul_type,arr_delay_min_gy,arr_delay_min_bin,arr_delay_ind,arr_delay
0,17.76531,38.01551,FLL,9,27.26231,157,1,33.52233,0,-15,-15,0,-15
1,9.88030,36.61623,SDF,16,36.13049,110,1,33.52233,0,45,45,1,35
2,11.70175,35.60278,MSY,22,42.37835,80,1,33.52233,0,-15,-15,0,-8
3,8.33723,44.79845,EWR,22,42.37835,166,1,33.52233,0,15,15,1,21
4,8.33723,36.41499,MCO,11,29.34927,141,1,33.52233,0,15,15,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,6.95741,36.12802,DSM,14,32.72667,85,1,33.52233,0,-15,-15,0,-2
159996,9.88030,33.54229,PHL,20,42.09990,88,1,33.52233,0,-45,-45,0,-31
159997,11.70175,37.05741,DAL,18,40.37584,90,1,33.52233,0,-15,-15,0,-14
159998,9.88030,39.75784,SFO,15,34.27183,408,1,33.52233,1,-45,-45,0,-41


In [15]:
df=df1[['carrier_v','dest_v','hh24_v', 'month_v', 'arr_delay' ]]
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)
X_train, y_train = train_df.drop(columns=["arr_delay"], axis=1), train_df["arr_delay"]
X_test, y_test = test_df.drop(columns=["arr_delay"], axis=1), test_df["arr_delay"]
lr = LogisticRegression()
lr.fit(X_train, y_train)



/Users/pelecheung/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
X_train.columns

Index(['carrier_v', 'dest_v', 'hh24_v', 'month_v'], dtype='object')

In [17]:
print("Model weights: %s"%(lr.coef_)) # these are weights
print("Model intercept: %s"%(lr.intercept_)) # this is the bias term
data = {'features': X_train.columns, 'coefficients':lr.coef_[0]}
pd.DataFrame(data)

Model weights: [[-0.0135634  -0.03565767 -0.02058012 -0.02973813]
 [-0.01384561 -0.03845276 -0.02845195 -0.0410788 ]
 [-0.01731301 -0.02593998 -0.0178276  -0.02629117]
 [-0.00745523 -0.03429229 -0.00111551 -0.04157309]
 [-0.0180875  -0.04506918 -0.01740483 -0.03746333]
 [-0.00914964 -0.0350193  -0.03597772 -0.0397787 ]
 [-0.02302989 -0.03822962  0.00937392 -0.04601293]
 [-0.00653822 -0.01043299 -0.03375426 -0.02700291]
 [-0.02829177 -0.01485245 -0.01407059 -0.03958854]
 [-0.02362604 -0.00344145 -0.03439713 -0.01390578]
 [-0.00774264 -0.02522395 -0.01215775 -0.02450542]
 [-0.02273154 -0.01863718  0.00468523 -0.03503389]
 [-0.0311322  -0.01627337  0.00334488 -0.03430614]
 [ 0.00356333 -0.0300949   0.00315844 -0.02905365]
 [-0.03058328 -0.02682216  0.00106694 -0.01354192]
 [-0.04805248  0.00967322 -0.00517586 -0.03262514]
 [-0.0492872  -0.01489282 -0.00541524  0.00310146]
 [-0.05029379  0.01681298 -0.02033207 -0.01053053]
 [-0.02202744  0.00776905 -0.01932346 -0.00436929]
 [-0.06606248 -0

,features,coefficients
0,carrier_v,-0.013563
1,dest_v,-0.035658
2,hh24_v,-0.020580
3,month_v,-0.029738


In [18]:
example = X_test.iloc[0,:]
example.tolist()

[11.70175, 37.28818, 27.87374, 33.52233]

In [19]:
lr.intercept_

array([-8.87847721e-04, -1.22600186e-03, -7.84717190e-04, -1.24066092e-03,
       -1.11817551e-03, -1.18714914e-03, -1.37306593e-03, -8.05945099e-04,
       -1.18136112e-03, -4.15104159e-04, -7.31350687e-04, -1.04543919e-03,
       -1.02371144e-03, -8.67001214e-04, -4.04196555e-04, -9.73534548e-04,
        9.24027386e-05, -3.14286728e-04, -1.30460040e-04,  9.30587085e-04,
        5.06976357e-04, -7.04719288e-06,  5.31312070e-04,  1.26604745e-03,
        1.15873532e-03,  1.90266161e-03,  1.71447380e-03,  2.21625643e-03,
        1.65214761e-03,  1.96737525e-03,  2.53434132e-03,  2.73261688e-03,
        2.69605262e-03,  3.06116369e-03,  3.60332699e-03,  3.33977931e-03,
        3.23912149e-03,  3.69900656e-03,  4.21277043e-03,  3.82936401e-03,
        3.92664787e-03,  3.70081267e-03,  3.72204982e-03,  3.83499548e-03,
        3.62190923e-03,  3.59659234e-03,  3.45368375e-03,  3.61668278e-03,
        3.62832052e-03,  3.56936106e-03,  3.51882250e-03,  3.11887655e-03,
        3.39205473e-03,  

In [20]:
lr.classes_

array([-60, -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48,
       -47, -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35,
       -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22,
       -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,
        -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,
         5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89])

## Start preidct

In [23]:
database = "mtpj.db"

conn = sqlite3.connect(database)

## For prove of concept use Flights_slim instead of Flights
sql_in="""SELECT cf.delay_pct as carrier_v, 
          --of.delay_pct as origin_v,
          df.delay_pct as dest_v,
          df.airport as dest,
          CAST(fl.crs_arr_hh24  as interger) crs_arr_hh24,
          hf.delay_pct  as                    hh24_v,
          --CAST(crs_elapsed_time as interger) crs_elapsed_time,
          CAST(fl.fl_month      as interger) month,
          mf.delay_pct  as                   month_v
          --CAST(haul_type        as interger) haul_type
          FROM  flights_test_7days fl, CARRIER_DELAY_F cf, AIRPORT_F df, AIRPORT_F of,
                HH24_F      hf, MONTH_F mf
          WHERE fl.mkt_unique_carrier = cf.mkt_unique_carrier
          AND   fl.dest               = df.airport
          AND   fl.origin             = of.airport
          AND   fl.crs_arr_hh24       = hf.crs_arr_hh24
          AND   fl.fl_month           = mf.fl_month
          ORDER BY RANDOM()
          LIMIT 160000;"""

df_stage = pd.read_sql(sql_in, con=conn)
conn.close()
df_stage
##df_stage=pd.read_csv('flights_f1.csv')

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,month,month_v
0,11.70175,38.88043,OKC,23,41.87090,1,33.52233
1,11.70175,36.27099,SAT,7,23.38308,1,33.52233
2,8.33723,36.48137,ORD,11,29.34927,1,33.52233
3,11.70175,34.66992,DEN,17,38.06039,1,33.52233
4,17.76531,37.71784,BOS,7,23.38308,1,33.52233
...,...,...,...,...,...,...,...
150618,8.33723,35.39247,IAH,0,41.03559,1,33.52233
150619,9.88030,34.22228,DCA,21,42.26300,1,33.52233
150620,8.33723,32.95197,CLT,23,41.87090,1,33.52233
150621,5.37583,24.07670,OTZ,8,24.03153,1,33.52233


In [24]:
df1=df_stage.dropna()
df1

,carrier_v,dest_v,dest,crs_arr_hh24,hh24_v,month,month_v
0,11.70175,38.88043,OKC,23,41.87090,1,33.52233
1,11.70175,36.27099,SAT,7,23.38308,1,33.52233
2,8.33723,36.48137,ORD,11,29.34927,1,33.52233
3,11.70175,34.66992,DEN,17,38.06039,1,33.52233
4,17.76531,37.71784,BOS,7,23.38308,1,33.52233
...,...,...,...,...,...,...,...
150618,8.33723,35.39247,IAH,0,41.03559,1,33.52233
150619,9.88030,34.22228,DCA,21,42.26300,1,33.52233
150620,8.33723,32.95197,CLT,23,41.87090,1,33.52233
150621,5.37583,24.07670,OTZ,8,24.03153,1,33.52233


In [25]:
dfp=df1[['carrier_v','dest_v','hh24_v', 'month_v' ]]
dfp


,carrier_v,dest_v,hh24_v,month_v
0,11.70175,38.88043,41.87090,33.52233
1,11.70175,36.27099,23.38308,33.52233
2,8.33723,36.48137,29.34927,33.52233
3,11.70175,34.66992,38.06039,33.52233
4,17.76531,37.71784,23.38308,33.52233
...,...,...,...,...
150618,8.33723,35.39247,41.03559,33.52233
150619,9.88030,34.22228,42.26300,33.52233
150620,8.33723,32.95197,41.87090,33.52233
150621,5.37583,24.07670,24.03153,33.52233


In [26]:
example = dfp.iloc[0,:]
example.tolist()
example.describe

<bound method NDFrame.describe of carrier_v    11.70175
dest_v       38.88043
hh24_v       41.87090
month_v      33.52233
Name: 0, dtype: float64>

In [27]:
lr.predict([example])

array([-9])

In [28]:
lr.predict_proba([example])

array([[9.95111098e-05, 4.37367150e-05, 1.75059079e-04, 1.71184996e-04,
        5.76944542e-05, 4.02513672e-05, 1.63635980e-04, 1.81886392e-04,
        1.77486591e-04, 2.95236986e-04, 2.71041274e-04, 4.17777403e-04,
        4.02159437e-04, 4.17333663e-04, 4.90014672e-04, 6.70012257e-04,
        8.34131729e-04, 9.58513393e-04, 1.20408100e-03, 1.31795249e-03,
        1.52212904e-03, 1.96020823e-03, 1.99233183e-03, 2.23967848e-03,
        2.45594829e-03, 2.87764737e-03, 3.53949260e-03, 3.35273496e-03,
        4.36476793e-03, 5.05663348e-03, 5.01072645e-03, 5.57030535e-03,
        6.38576836e-03, 7.61595690e-03, 8.67611906e-03, 9.84717723e-03,
        1.09927947e-02, 1.17634551e-02, 1.28278595e-02, 1.50999869e-02,
        1.61224425e-02, 1.78792283e-02, 2.01850665e-02, 2.00086580e-02,
        2.43114807e-02, 2.40496046e-02, 2.47352866e-02, 2.47830762e-02,
        2.45201941e-02, 2.52343949e-02, 2.50249597e-02, 2.65145712e-02,
        2.46821657e-02, 2.40991118e-02, 2.40255367e-02, 2.229507

In [32]:
data_dict = {"predicted": lr.predict(dfp).tolist()}
pd.DataFrame(data_dict)
out_file=pd.DataFrame(data_dict)
out_file

,predicted
0,-9
1,-12
2,-12
3,-11
4,-10
...,...
150618,-13
150619,-13
150620,-13
150621,-12


In [34]:
out_file.to_csv('predict.csv')

In [242]:
## Test
dfp.to_csv('first_part.csv')